General information
------------------

Source (organization): ESnet

Contacts: ESnet (Eric Pouyoul)

Time coverage: 8/1/2015 until 1/16/2016

Features type: TSTAT

Access control: public

Known users: X-SWAP


Overview
--------

This data set consists of [TSTAT](http://tstat.polito.it/) (TCP STatistic and Analysis Tool) data collected at NERSC Data Transfer Nodes (DTNs). The TSTAT tool captures network measurement data for both forward (from NERSC DTN) and backward (to NERSC DTN) stream of packets. TSTAT data is related to the traditional network flow data but differs because it is collected at the source endpoint. Due to this quality, TSTAT data is easier to correlate with other measurements performed on the hosts, such as application logs, I/O etc. The measurement data is collected in a set of text files where each row corresponds to a different flow and each column is associated to a specific measurement. More information about these files and specific measurements can be found [here](http://tstat.tlc.polito.it/measure.shtml#LOG). 

Data Collection and Visualization
---------

The TSTAT measurement files are created periodically (by default every 5 minutes) on each DTN, and are processed every hour through a client software, [tstat-transport](https://github.com/esnet/tstat-transport), that converts the data into JSON objects and sends them to a RabbitMQ server. There is a [logstash](https://www.elastic.co/guide/en/logstash/current/introduction.html) instance running at NERSC that pulls the data from the RabbitMQ server, calculates additional measurements (like throughput, packets-retransmit% etc.), and ingests them into [Elasticsearch](https://www.elastic.co/guide/en/elasticsearch/reference/current/getting-started.html). NERSC provides a [Kibana](https://www.elastic.co/guide/en/kibana/current/introduction.html) portal to search and visualize the data from Elasticsearch.

In order to visualize TSTAT data, users need to select the appropriate Elasticsearch index, i.e., ** dtn-tstat-2017* **. In order to generate graphs, users can click on the * Visualize * option and select the appropriate graph type. Additionally, users can also create a dashboard with a collection of multiple graphs by selecting the * Dashboard * option on the left menu.

Access
------

The Kibana portal at NERSC can be accessed via the following url (https://kb.nersc.gov). Users need to have a NERSC NIM account to be able to access the data. 

Statistical Analysis
----------
In addition to the Kibana visualization, more sophisticated and complex statistical can also be done using the TSTAT data stored in Elasticsearch. We have developed an initial prototype that retrieves the data from Elasticsearch and allows users to perform complex queries and graph analysis on the data. It adds several aggregated columns and cleans the data prior to ingesting it in a MySQL database. This pre-processed data is then used for statistical analysis and building prediction models.

Similar frameworks or tools can be used to do statistical data analysis interactively through the interface. An example is shown below.

#### Sample Interface

Get the RTT values from TSTAT data for some endpoints.

In [ ]:
# this is sample code quering the data set
import tstat_analyzer as analyzer

analyzer.get_rtt_values()

Retrieve network performance metrics for some NERSC DTNs.

In [ ]:
analyzer.get_metrics()

Results, issues, filtering, pre-processing
----------------------------------------

X-SWAP initially used this data set for.. needed filtering on .. because.. initial promissing results are...


Schema
------

This dataset contains the following features...